In [41]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from concurrent.futures import ThreadPoolExecutor
import os

import numpy as np
import pandas as pd
import sklearn
from tqdm.auto import tqdm

In [43]:
%cd drive/MyDrive/Colab Notebooks/cs230/

/content/drive/MyDrive/Colab Notebooks/cs230


In [70]:
dataset_root_dir = '/content/drive/MyDrive/Colab Notebooks/cs230/sustainbench/dataset_preprocessing/dhs_lsms'

In [109]:
df = pd.read_csv(os.path.join(dataset_root_dir, 'output_labels/dhs_final_labels.csv'))
df['survey'] = df['DHSID_EA'].str[:10]
df['cc'] = df['DHSID_EA'].str[:2]
df['path'] = dataset_root_dir + '/dhs_npzs/' + df['survey'] + '/' + df['DHSID_EA'] + '.npz'

In [112]:
# subset a dataset with just 'ZM'
df_zm = df[ df['cc']=='ZM' ]
zm_path_years = df_zm[['DHSID_EA', 'path', 'year']].apply(tuple, axis=1)
df_zm.set_index('DHSID_EA', verify_integrity=True, inplace=True)

In [113]:
df_zm.head()

,cname,year,lat,lon,n_asset,asset_index,n_water,water_index,n_sanitation,sanitation_index,...,women_bmi,n_women_edu,n_women_bmi,cluster_id,adm1fips,adm1dhs,urban,survey,cc,path
DHSID_EA,,,,,,,,,,,,,,,,,,,,,
ZM-2007-5#-00000001,ZM,2007,-16.795975,26.975610,23.0,2.806357,23.0,5.000000,23.0,4.913043,...,25.182500,30.0,28.0,1,ZA04,8,U,ZM-2007-5#,ZM,/content/drive/MyDrive/Colab Notebooks/cs230/s...
ZM-2007-5#-00000002,ZM,2007,-13.036608,26.111080,6.0,-2.626596,6.0,2.500000,6.0,1.833333,...,NaN,5.0,NaN,2,ZA01,7,R,ZM-2007-5#,ZM,/content/drive/MyDrive/Colab Notebooks/cs230/s...
ZM-2007-5#-00000003,ZM,2007,-14.248170,31.336041,23.0,1.577803,23.0,5.000000,23.0,3.565217,...,22.670333,33.0,30.0,3,ZA06,3,U,ZM-2007-5#,ZM,/content/drive/MyDrive/Colab Notebooks/cs230/s...
ZM-2007-5#-00000004,ZM,2007,-10.220802,31.149618,22.0,-0.226965,22.0,3.954545,22.0,3.363636,...,22.109000,32.0,30.0,4,ZA07,6,U,ZM-2007-5#,ZM,/content/drive/MyDrive/Colab Notebooks/cs230/s...
ZM-2007-5#-00000005,ZM,2007,-14.376148,23.285022,19.0,-2.521934,19.0,3.526316,19.0,1.263158,...,21.828333,22.0,18.0,5,ZA03,9,R,ZM-2007-5#,ZM,/content/drive/MyDrive/Colab Notebooks/cs230/s...


In [114]:
zm_path_years

114662    (ZM-2007-5#-00000001, /content/drive/MyDrive/C...
114663    (ZM-2007-5#-00000002, /content/drive/MyDrive/C...
114664    (ZM-2007-5#-00000003, /content/drive/MyDrive/C...
114665    (ZM-2007-5#-00000004, /content/drive/MyDrive/C...
114666    (ZM-2007-5#-00000005, /content/drive/MyDrive/C...
                                ...                        
116230    (ZM-2018-7#-00000541, /content/drive/MyDrive/C...
116231    (ZM-2018-7#-00000542, /content/drive/MyDrive/C...
116232    (ZM-2018-7#-00000543, /content/drive/MyDrive/C...
116233    (ZM-2018-7#-00000544, /content/drive/MyDrive/C...
116234    (ZM-2018-7#-00000545, /content/drive/MyDrive/C...
Length: 1573, dtype: object

In [75]:
path_years = df[['DHSID_EA', 'path', 'year']].apply(tuple, axis=1)

KeyError: ignored

In [74]:
df.set_index('DHSID_EA', verify_integrity=True, inplace=True)
print(df['path'].iloc[0])
display(df.head())

/content/drive/MyDrive/Colab Notebooks/cs230/sustainbench/dataset_preprocessing/dhs_lsms/dhs_npzs/AL-2008-5#/AL-2008-5#-00000001.npz


,cname,year,lat,lon,n_asset,asset_index,n_water,water_index,n_sanitation,sanitation_index,...,women_bmi,n_women_edu,n_women_bmi,cluster_id,adm1fips,adm1dhs,urban,survey,cc,path
DHSID_EA,,,,,,,,,,,,,,,,,,,,,
AL-2008-5#-00000001,AL,2008,40.822652,19.838321,18.0,2.430596,18.0,3.444444,18.0,4.833333,...,24.365000,18.0,18.0,1,NaN,9999,R,AL-2008-5#,AL,/content/drive/MyDrive/Colab Notebooks/cs230/s...
AL-2008-5#-00000002,AL,2008,40.696846,20.007555,20.0,2.867678,20.0,4.700000,20.0,4.950000,...,23.104000,20.0,20.0,2,NaN,9999,R,AL-2008-5#,AL,/content/drive/MyDrive/Colab Notebooks/cs230/s...
AL-2008-5#-00000003,AL,2008,40.750037,19.974262,18.0,2.909049,18.0,4.500000,18.0,4.777778,...,22.387778,18.0,18.0,3,NaN,9999,R,AL-2008-5#,AL,/content/drive/MyDrive/Colab Notebooks/cs230/s...
AL-2008-5#-00000004,AL,2008,40.798931,19.863338,19.0,2.881122,19.0,4.947368,19.0,4.789474,...,27.084500,21.0,20.0,4,NaN,9999,R,AL-2008-5#,AL,/content/drive/MyDrive/Colab Notebooks/cs230/s...
AL-2008-5#-00000005,AL,2008,40.746123,19.843885,19.0,2.546830,19.0,4.684211,19.0,4.526316,...,24.523125,16.0,16.0,5,NaN,9999,R,AL-2008-5#,AL,/content/drive/MyDrive/Colab Notebooks/cs230/s...


In [150]:
label_cols = ['asset_index', 'under5_mort', 'women_edu', 'water_index', 'sanitation_index']

In [ ]:
def calculate_nl_mean(path_and_year):
    '''
    Args
    - path_year: tuple (path, year)
      - path: str, path to npz file containing single entry 'x'
        representing a (C, H, W) image
      - year: int

    Returns: (nl_mean, year)
    '''
    dhsid_ea, npz_path, year = path_and_year
    img = np.load(npz_path)['x']  # shape (C, H, W)
    nl_mean = img[-1].mean(dtype=np.float64)

    #print('npz_path is ', npz_path)
    #print('year is ', year)
    #print('nl_mean is ', nl_mean)
    
    return dhsid_ea, nl_mean, year

In [79]:
df_zm[label_cols].head()

,asset_index,under5_mort,women_bmi,women_edu,water_index,sanitation_index
114662,2.806357,0.000000,25.182500,10.733333,5.000000,4.913043
114663,-2.626596,0.000000,NaN,4.600000,2.500000,1.833333
114664,1.577803,83.333333,22.670333,11.272727,5.000000,3.565217
114665,-0.226965,173.913043,22.109000,7.687500,3.954545,3.363636
114666,-2.521934,0.000000,21.828333,5.818182,3.526316,1.263158


In [55]:
results_df = pd.DataFrame(
    data=np.nan,
    columns=['nl_mean', 'year'],
    index=pd.Index(df.index, name='DHSID_EA')
)
results_df.head()

,nl_mean,year
DHSID_EA,,
AL-2008-5#-00000001,NaN,NaN
AL-2008-5#-00000002,NaN,NaN
AL-2008-5#-00000003,NaN,NaN
AL-2008-5#-00000004,NaN,NaN
AL-2008-5#-00000005,NaN,NaN


In [118]:
results_df_zm = pd.DataFrame(
    data=np.nan,
    columns=['nl_mean', 'year'],
    index=pd.Index(df_zm.index.tolist(), name='DHSID_EA')
)
results_df_zm.head()

,nl_mean,year
DHSID_EA,,
ZM-2007-5#-00000001,NaN,NaN
ZM-2007-5#-00000002,NaN,NaN
ZM-2007-5#-00000003,NaN,NaN
ZM-2007-5#-00000004,NaN,NaN
ZM-2007-5#-00000005,NaN,NaN


In [119]:
with ThreadPoolExecutor(max_workers=30) as pool:
    inputs = zm_path_years
    futures = pool.map(calculate_nl_mean, inputs)
    for dhsid_ea, nl_mean, year in tqdm(futures, total=len(inputs)):
        #print(dhsid_ea)
        #print(nl_mean)
        #print(year)
        results_df_zm.loc[dhsid_ea, ['nl_mean', 'year']] = (nl_mean, year)

  0%|          | 0/1573 [00:00<?, ?it/s]

In [120]:
results_df_zm.to_csv('/content/drive/MyDrive/Colab Notebooks/cs230/sustainbench/baseline_models/dhs/mean_nl.csv')

In [121]:
results_df_zm.head()

,nl_mean,year
DHSID_EA,,
ZM-2007-5#-00000001,14.460893,2007.0
ZM-2007-5#-00000002,0.000000,2007.0
ZM-2007-5#-00000003,6.420197,2007.0
ZM-2007-5#-00000004,6.776630,2007.0
ZM-2007-5#-00000005,0.391391,2007.0


In [122]:
results_df_zm['year'] = results_df_zm['year'].astype(int)

In [ ]:
SPLITS = {
    'train': [
        'AL', 'BD', 'CD', 'CM', 'GH', 'GU', 'HN', 'IA', 'ID', 'JO', 'KE', 'KM',
        'LB', 'LS', 'MA', 'MB', 'MD', 'MM', 'MW', 'MZ', 'NG', 'NI', 'PE', 'PH',
        'SN', 'TG', 'TJ', 'UG', 'ZM', 'ZW'],
    'val': [
        'BF', 'BJ', 'BO', 'CO', 'DR', 'GA', 'GN', 'GY', 'HT', 'NM', 'SL', 'TD',
        'TZ'],
    'test': [
        'AM', 'AO', 'BU', 'CI', 'EG', 'ET', 'KH', 'KY', 'ML', 'NP', 'PK', 'RW',
        'SZ']
}
SPLITS['trainval'] = SPLITS['train'] + SPLITS['val']

In [139]:
SPLITS = {
    'train': [ 2007 ],
    'val': [ 2013 ],
    'test': [ 2018 ]
}
SPLITS['trainval'] = SPLITS['train'] + SPLITS['val']

In [126]:
import scipy.stats
import sklearn.neighbors

In [124]:
results_df['cc'] = results_df.index.str[:2]

In [127]:
df = df_zm 
results_df = results_df_zm

In [131]:
df.head()

,cname,year,lat,lon,n_asset,asset_index,n_water,water_index,n_sanitation,sanitation_index,...,women_bmi,n_women_edu,n_women_bmi,cluster_id,adm1fips,adm1dhs,urban,survey,cc,path
DHSID_EA,,,,,,,,,,,,,,,,,,,,,
ZM-2007-5#-00000001,ZM,2007,-16.795975,26.975610,23.0,2.806357,23.0,5.000000,23.0,4.913043,...,25.182500,30.0,28.0,1,ZA04,8,U,ZM-2007-5#,ZM,/content/drive/MyDrive/Colab Notebooks/cs230/s...
ZM-2007-5#-00000002,ZM,2007,-13.036608,26.111080,6.0,-2.626596,6.0,2.500000,6.0,1.833333,...,NaN,5.0,NaN,2,ZA01,7,R,ZM-2007-5#,ZM,/content/drive/MyDrive/Colab Notebooks/cs230/s...
ZM-2007-5#-00000003,ZM,2007,-14.248170,31.336041,23.0,1.577803,23.0,5.000000,23.0,3.565217,...,22.670333,33.0,30.0,3,ZA06,3,U,ZM-2007-5#,ZM,/content/drive/MyDrive/Colab Notebooks/cs230/s...
ZM-2007-5#-00000004,ZM,2007,-10.220802,31.149618,22.0,-0.226965,22.0,3.954545,22.0,3.363636,...,22.109000,32.0,30.0,4,ZA07,6,U,ZM-2007-5#,ZM,/content/drive/MyDrive/Colab Notebooks/cs230/s...
ZM-2007-5#-00000005,ZM,2007,-14.376148,23.285022,19.0,-2.521934,19.0,3.526316,19.0,1.263158,...,21.828333,22.0,18.0,5,ZA03,9,R,ZM-2007-5#,ZM,/content/drive/MyDrive/Colab Notebooks/cs230/s...


In [132]:
results_df.head()

,nl_mean,year
DHSID_EA,,
ZM-2007-5#-00000001,14.460893,2007
ZM-2007-5#-00000002,0.000000,2007
ZM-2007-5#-00000003,6.420197,2007
ZM-2007-5#-00000004,6.776630,2007
ZM-2007-5#-00000005,0.391391,2007


In [149]:
df.index[df['year'].isin(SPLITS['train'])  & df['women_bmi'].notna()]

Index(['ZM-2007-5#-00000001', 'ZM-2007-5#-00000003', 'ZM-2007-5#-00000004',
       'ZM-2007-5#-00000005', 'ZM-2007-5#-00000006', 'ZM-2007-5#-00000007',
       'ZM-2007-5#-00000008', 'ZM-2007-5#-00000009', 'ZM-2007-5#-00000010',
       'ZM-2007-5#-00000011',
       ...
       'ZM-2007-5#-00000311', 'ZM-2007-5#-00000312', 'ZM-2007-5#-00000313',
       'ZM-2007-5#-00000314', 'ZM-2007-5#-00000315', 'ZM-2007-5#-00000316',
       'ZM-2007-5#-00000317', 'ZM-2007-5#-00000318', 'ZM-2007-5#-00000319',
       'ZM-2007-5#-00000320'],
      dtype='object', name='DHSID_EA', length=313)

In [142]:
def run(knn, label, dmsp, trainsplit='train', testsplit='test'):
    #if dmsp:
    #    year_mask = (df['year'] <= 2011)
    #else:
    #    year_mask = (df['year'] > 2011)

    train_dhsids = df.index[df['year'].isin(SPLITS[trainsplit]) & df[label].notna()]
    test_dhsids = df.index[df['year'].isin(SPLITS[testsplit]) & df[label].notna()]
    #print(train_dhsids)
    #print(test_dhsids)

    train_X = results_df.loc[train_dhsids, 'nl_mean'].values.reshape(-1, 1)
    train_Y = df.loc[train_dhsids, label].values
    test_X = results_df.loc[test_dhsids, 'nl_mean'].values.reshape(-1, 1)
    test_Y = df.loc[test_dhsids, label].values

    knn.fit(train_X, train_Y)
    preds = knn.predict(test_X)
    return preds, test_Y

In [151]:
for label in label_cols:
    print(f'=========== {label:15s} ============')
    best_r2 = 0
    best_k = None
    for k in range(1, 21):
        knn = sklearn.neighbors.KNeighborsRegressor(
            n_neighbors=k, weights='uniform', algorithm='auto')
        preds_dmsp, labels_dmsp = run(knn, label, True, 'train', 'val')
        preds_viirs, labels_viirs = run(knn, label, False, 'train', 'val')
        r2 = scipy.stats.pearsonr(
            np.concatenate([preds_dmsp, preds_viirs]),
            np.concatenate([labels_dmsp, labels_viirs])
        )[0]**2
        print(f'k={k:2d}, {label:15s} r^2 = {r2:.3f}')
        if r2 > best_r2:
            best_r2 = r2
            best_k = k
    knn = sklearn.neighbors.KNeighborsRegressor(
            n_neighbors=best_k, weights='uniform', algorithm='auto')
    preds_dmsp, labels_dmsp = run(knn, label, True, 'trainval', 'test')
    preds_viirs, labels_viirs = run(knn, label, False, 'trainval', 'test')
    r2 = scipy.stats.pearsonr(
        np.concatenate([preds_dmsp, preds_viirs]),
        np.concatenate([labels_dmsp, labels_viirs])
    )[0]**2
    print(f'FINAL: k={best_k:2d}, {label:15s} r^2 = {r2:.2f}')

=========== asset_index     ============
k= 1, asset_index     r^2 = 0.390
k= 2, asset_index     r^2 = 0.460
k= 3, asset_index     r^2 = 0.504
k= 4, asset_index     r^2 = 0.543
k= 5, asset_index     r^2 = 0.548
k= 6, asset_index     r^2 = 0.560
k= 7, asset_index     r^2 = 0.563
k= 8, asset_index     r^2 = 0.589
k= 9, asset_index     r^2 = 0.593
k=10, asset_index     r^2 = 0.603
k=11, asset_index     r^2 = 0.609
k=12, asset_index     r^2 = 0.605
k=13, asset_index     r^2 = 0.616
k=14, asset_index     r^2 = 0.614
k=15, asset_index     r^2 = 0.608
k=16, asset_index     r^2 = 0.619
k=17, asset_index     r^2 = 0.611
k=18, asset_index     r^2 = 0.611
k=19, asset_index     r^2 = 0.612
k=20, asset_index     r^2 = 0.610
FINAL: k=16, asset_index     r^2 = 0.64
=========== under5_mort     ============
k= 1, under5_mort     r^2 = 0.004
k= 2, under5_mort     r^2 = 0.000
k= 3, under5_mort     r^2 = 0.000
k= 4, under5_mort     r^2 = 0.000
k= 5, under5_mort     r^2 = 0.003
k= 6, under5_mort     r^2 = 